---
title: "A look at different scenarios - Gains and Losses"
format:
  html:
    code-fold: true
    code-summary: "Show the code"
  docx: 
    fig-height: 11.6 in
---

In [52]:
import pandas as pd
from fagrants_module import fagrants

def weighted_group_delta(grouped_df):
    return_dict = {}

    for year in range(2025, 2029):
        total_weight = grouped_df[f"ERP_{year}"].mean()

        return_dict[f"Population_{year}"] = total_weight

        weighted_mean = (
            grouped_df[f"Grant Delta_{year}"] * grouped_df[f"ERP_{year}"]
        ).sum() / total_weight

        return_dict[f"Grant Delta_{year}"] = weighted_mean

    return pd.Series(return_dict)

def grants_simulation(data, pc_min, pc_min_base):
    inst = fagrants.fagrants_model(
    data_path=data,
    per_capita_minimum=pc_min,
    per_capita_minimum_base=pc_min_base,
    )

    sim_grants = inst.run()

    sim_grants["Sim_i"] = sim_grants["UID"].str.split("_").str[1].astype(int)

    for year in range(2025, 2029):
        sim_grants[f"Grant Delta_{year}"] = (
            sim_grants[f"Grant_{year}"] - sim_grants[f"Grant_base_{year}"]
        )

    return sim_grants

def grants_delta(sim_grants):
    sim_grants_grouped = (
        sim_grants.groupby(["Jurisdiction", "ACLG", "LGA"])
        .apply(weighted_group_delta, include_groups=False)
        .reset_index()
    )

    pop_melted = pd.melt(
        sim_grants_grouped,
        id_vars=["Jurisdiction","ACLG", "LGA"],
        value_vars=[f"Population_{year}" for year in range(2025, 2029)],
        var_name="Year",
        value_name="Population",
    )

    sim_grants_melted = pd.melt(
        sim_grants_grouped,
        id_vars=["Jurisdiction","ACLG", "LGA"],
        value_vars=[f"Grant Delta_{year}" for year in range(2025, 2029)],
        var_name="Year",
        value_name="Expected Change in GPG ($)",
    )

    for year in range(2025, 2029):
        sim_grants_grouped[f"Grant Delta Per Capita_{year}"] = sim_grants_grouped[f"Grant Delta_{year}"]/ sim_grants_grouped[f"Population_{year}"]

    sim_grants_melted_per_capita = pd.melt(
        sim_grants_grouped,
        id_vars=["Jurisdiction", "ACLG", "LGA"],
        value_vars=[f"Grant Delta Per Capita_{year}" for year in range(2025, 2029)],
        var_name="Year",
        value_name="Expected Change in GPG Per Capita($)",
    )

    sim_grants_melted["Year"] = sim_grants_melted["Year"].str.split("_").str[1].astype(int)

    sim_grants_melted["Expected Change in GPG Per Capita($)"] = sim_grants_melted_per_capita["Expected Change in GPG Per Capita($)"]

    sim_grants_melted["Population"] = pop_melted["Population"]

    return sim_grants_melted

In [46]:
sim_grants_30 = grants_simulation("./Data/FA Grants Tables - Python.xlsx",
    0.30, 0.30)

sim_grants_20 = grants_simulation("./Data/FA Grants Tables - Python.xlsx",
    0.20, 0.30)

sim_grants_10 = grants_simulation("./Data/FA Grants Tables - Python.xlsx",
    0.10, 0.30)

sim_grants_0 = grants_simulation("./Data/FA Grants Tables - Python.xlsx",
    0.00, 0.30)

In [53]:
sim_grants_30_melted = grants_delta(sim_grants_30)

sim_grants_20_melted = grants_delta(sim_grants_20)

sim_grants_10_melted = grants_delta(sim_grants_10)

sim_grants_0_melted = grants_delta(sim_grants_0)

In [54]:
from datetime import datetime as tm

with pd.ExcelWriter(f"./Data/Output/Simulation Results {tm.today().strftime("%d_%m_%Y")}.xlsx") as writer:
    sim_grants_30_melted.to_excel(writer, sheet_name = "30% Minimum Grant")
    sim_grants_20_melted.to_excel(writer, sheet_name = "20% Minimum Grant")
    sim_grants_10_melted.to_excel(writer, sheet_name = "10% Minimum Grant")
    sim_grants_0_melted.to_excel(writer, sheet_name = "0% Minimum Grant")

In [58]:
non_key_columns = [i for i in list(sim_grants_20_melted.columns) if i not in ["Jurisdiction", "ACLG", "LGA", "Year"]]

## A 20% Per Capita Minimum Grant

In [64]:
#for state in sim_grants_20_melted["Jurisdiction"].unique():
sub = sim_grants_20_melted[sim_grants_20_melted["Jurisdiction"] == "Tasmania"]

def weighted_mean(x): return sum(x * sub["Population"]) / sub["Population"].sum()

sub.groupby(["Jurisdiction", "ACLG", "Year"])[non_key_columns].agg(
    {
        "Expected Change in GPG ($)": weighted_mean,
        "Expected Change in GPG Per Capita($)": weighted_mean,
        "Population": 'sum'
    }
)

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Tasmania     RAL  2025                         NaN   
                  2026                         NaN   
                  2027                         NaN   
                  2028                         NaN   
             RAM  2025                         NaN   
                  2026                         NaN   
                  2027                         NaN   
                  2028                         NaN   
             RAS  2025                         NaN   
                  2026                         NaN   
                  2027                         NaN   
                  2028                         NaN   
             RAV  2025                         NaN   
                  2026                         NaN   
                  2027                         NaN   
                  2028                         NaN   
             UCC  2025                         NaN   
                  2026                         NaN   
                  2027                         NaN   
                  2028                         NaN   
             UFM  2025                         NaN   
                  2026                         NaN   
                  2027                         NaN   
                  2028                         NaN   
             UFS  2025                         NaN   
                  2026                         NaN   
                  2027                         NaN   
                  2028                         NaN   
             URM  2025                         NaN   
                  2026                         NaN   
                  2027                         NaN   
                  2028                         NaN   
             URS  2025                         NaN   
                  2026                         NaN   
                  2027                         NaN   
                  2028                         NaN   

                        Expected Change in GPG Per Capita($)  Population  
Jurisdiction ACLG Year                                                    
Tasmania     RAL  2025                                   NaN     48472.0  
                  2026                                   NaN     48641.0  
                  2027                                   NaN     48810.0  
                  2028                                   NaN     48978.0  
             RAM  2025                                   NaN     27678.0  
                  2026                                   NaN     27773.0  
                  2027                                   NaN     27869.0  
                  2028                                   NaN     27966.0  
             RAS  2025                                   NaN      3122.0  
                  2026                                   NaN      3133.0  
                  2027                                   NaN      3144.0  
                  2028                                   NaN      3154.0  
             RAV  2025                                   NaN    113763.0  
                  2026                                   NaN    114158.0  
                  2027                                   NaN    114553.0  
                  2028                                   NaN    114948.0  
             UCC  2025                                   NaN     53264.0  
                  2026                                   NaN     53449.0  
                  2027                                   NaN     53634.0  
                  2028                                   NaN     53819.0  
             UFM  2025                                   NaN    149166.0  
                  2026                                   NaN    149683.0  
                  2027                                   NaN    150201.0  
                  2028                                   NaN    150718.0  
             UFS  202

In [61]:
k

Expected Change in GPG ($)  \
Jurisdiction      ACLG Year                               
Western Australia RAL  2025                         NaN   
                       2026                         NaN   
                       2027                         NaN   
                       2028                         NaN   
                  RAM  2025                         NaN   
...                                                 ...   
                  URM  2028                         NaN   
                  URS  2025                         NaN   
                       2026                         NaN   
                       2027                         NaN   
                       2028                         NaN   

                             Expected Change in GPG Per Capita($)  Population  
Jurisdiction      ACLG Year                                                    
Western Australia RAL  2025                                   NaN     63674.0  
                       2026                                   NaN     64726.0  
                       2027                                   NaN     65630.0  
                       2028                                   NaN     66576.0  
                  RAM  2025                                   NaN     47913.5  
...                                                           ...         ...  
                  URM  2028                                   NaN    207928.5  
                  URS  2025                                   NaN     52713.0  
                       2026                                   NaN     53583.0  
                       2027                                   NaN     54331.0  
                       2028                                   NaN     55114.0  

[80 rows x 3 columns]

## A 10% Per Capita Minimum Grant

## A World With No Per Capita Minimum Grant

## And for sanity reasons, status quo